In [1]:
# Import required libraries for data manipulation and analysis
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr

In [2]:
#Import required sklearn functions
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn.inspection import permutation_importance
from collections import defaultdict

In [3]:
#Import sklearn classifiers
from sklearn.ensemble import RandomForestClassifier

In [4]:
#Import library to oversample 
from imblearn.over_sampling import RandomOverSampler

In [5]:
#Import RDKit and Mordred libraries
from rdkit import Chem
from rdkit.Chem import Draw
from mordred import Calculator, descriptors

In [6]:
# Sets Pandas Display to Monitor Code
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [7]:
# Create Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True)

In [8]:
#Functions used in the study

#Remove those numbers from analysis data
def filter_rows_by_values1(df, col, values):
    return df[~df[col].isin(values)]

#Remove those numbers from analysis data
def filter_rows_by_values2(df, col, values):
    return df[df[col].isin(values)]

#Get Mordred calcs
def get_Mordred(data_input):
    # Assigns Reactants Mordred Info
    reactants = data_input['Substrate']
    
    reactants_mol_list = []
    for inChi_reactants in reactants:
      reactants_mol = Chem.MolFromInchi(inChi_reactants)
      reactants_mol_list.append(reactants_mol)

    # Puts reactants into Pandas Type
    reactant_data = []
    reactant_data = calc.pandas(reactants_mol_list)
       
    #Joins Mordred parameters with experimental, atomic charges, and JChem for Excel parameters
    add_reactants = pd.concat((data_input, reactant_data), axis=1)
    
    #Force any non-numeric entries as NaN and replace them with 0
    int_data = add_reactants.apply(pd.to_numeric, errors='coerce')
    
    output = int_data.fillna(0)#, inplace=True)

    return output

#Remove zero varience
def remove_zero_varience(values):
   sel = VarianceThreshold()
   _ = sel.fit(values)
   mask = sel.get_support()
   values = values.loc[:,mask] 
   return values

def remove_95correlated(correlated):
    #Remove any features that are greater than 95% correlated
    corr_matrix = correlated.corr()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

    correlated = correlated.drop(to_drop, axis = 1)
    corr_matrix = correlated.corr()
    return correlated

def remove_nonimportant(X_values, y_values):
    # Specifys Random Forest and the Number of Trees, SelectFromModel will
    # select features which are most important
    feature_names = [f"feature {i}" for i in range(X_values.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(X_values, y_values)

    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time

    threshold = np.sort(importances)[-100]
    
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(X_values, y_values)

    # Select the final features set 
    sel.get_support()
    selected_feat= X_values.columns[(sel.get_support())]

    # Prints the names of the final selected features
    print(selected_feat)
    X_values = X_values[selected_feat]
    
    return X_values

def dendrogram(X_values, y):
    corr = spearmanr(X_values).correlation
    # Ensure the correlation matrix is symmetric
    corr = (corr + corr.T) / 2
    np.fill_diagonal(corr, 1)
    distance_matrix = 1 - np.abs(corr)
    dist_linkage = hierarchy.ward(squareform(distance_matrix))
  
    trained_cluster_ids = hierarchy.fcluster(dist_linkage, y, criterion="distance")
    trained_cluster_id_to_feature_ids = defaultdict(list) 
    for idx, trained_cluster_id in enumerate(trained_cluster_ids):
        trained_cluster_id_to_feature_ids[trained_cluster_id].append(idx)
    
    trained_selected_features = [v[0] for v in trained_cluster_id_to_feature_ids.values()]
    final_selected_features = X_values.columns[trained_selected_features]
    X_train = X_values[final_selected_features]
    return X_train

def classificationMetrics(results, y_test, pred):
    acc = accuracy_score(y_test, pred)
    prec = precision_score(y_test, pred, average=None, zero_division=0)
    recall = recall_score(y_test, pred, average=None)
    F1 = f1_score(y_test, pred, average=None)           
    #Calculate confusion matrix
    cf_matrix = confusion_matrix(y_test, pred)
    cf_matrix = np.reshape(cf_matrix,(1,4))
    comb = np.concatenate((x, y, cf_matrix, acc, prec, recall, F1), axis=None)
    comb = [comb]
    results = results.append(pd.DataFrame(comb, columns=results.columns), ignore_index=True)
    return results


In [9]:
# Read Training/Test data input File
data = pd.read_csv('BorylationTrainingTest 9-12-24.csv')

#group the compounds by numbers
data['grouped'] = data.groupby('Substrate', sort=False).ngroup()

# Seperate dataset as response variable and feature variables
data = data.drop(['Boronic Ester','Active Catalyst-Ligand','Catalyst','Ligand','Solvent',
                 'Temp','Buried_Vol','PyramidalizationAR','PyramidalizationG','SASA_area', 'SASA_vol','Sterimol_L',
                 'SterimolB_1','Sterimol_B_5','Buried_Sterimol_L','Buried_SterimolB_1','Buried_Sterimol_B_5','Product',
                 'Substrate',],axis = 1)


In [10]:
Results_df = pd.DataFrame(columns =  ['x', 'y',  "True Neg","False Pos","False Neg","True Pos",'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

maxacc_comb = pd.DataFrame()
val_tot = pd.DataFrame()
prod = pd.DataFrame()
test_index_total = pd.DataFrame()

model_columns = pd.DataFrame()
for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 200,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(data, "grouped", set_numbers)
    training_data = training_data.drop('grouped', axis = 1)    
    test_data = filter_rows_by_values2(data, "grouped", set_numbers)
    test_data = test_data.drop('grouped', axis = 1)    
    
    #Remove features that dont change
    training_data = remove_zero_varience(training_data)
    
    #Remove features that are more than 95% correlated
    training_data = remove_95correlated(training_data)
    
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio', axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio', axis = 1)
    test_y = test_data['Product_Ratio']
 
    #Remove features that are considered less important
    feature_names = [f"feature {i}" for i in range(training_X.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(training_X, training_y)
    
    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time
    
    threshold = np.sort(importances)[-35] 
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(training_X, training_y)
     
    # Select the reduced features set 
    sel.get_support()
    selected_feat= training_X.columns[(sel.get_support())]
    
    reduced1_X = training_X[selected_feat]
    test_X = test_X[selected_feat]
    
    #Apply over-sampling to dataset
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(reduced1_X, training_y) 
    
    for y in [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
              0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 
              0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35,
              0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 
              0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45,
              0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50,
              0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55]:
             
    
        #Make final training and test set and save them as df's  
        X_train = dendrogram(X_resampled, y)
        test_X = test_X[X_train.columns]
        training_columns_list = X_train.columns.tolist()
        training_columns_list = (x, y, training_columns_list)
        training_columns_list = (pd.DataFrame(training_columns_list).T)

        #Random Forest Classifier
        rfc = RandomForestClassifier(n_estimators=800,max_depth=9)
        rfc.fit(X_train, y_resampled)
        pred = rfc.predict(test_X)
        Results_df = classificationMetrics(Results_df, test_y, pred)

        #Evaluate model by going line by line
        ynew = rfc.predict(test_X)
        prediction_df = pd.DataFrame(ynew,  columns = [(x,y)])

        val_pred_T = prediction_df.T
        val_tot = val_tot.append(val_pred_T)

        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = Results_df.groupby('y')['acc'].mean()
        acc_std = Results_df.groupby('y')['acc'].std()
        precision_0_mean = Results_df.groupby('y')['precision 0'].mean()
        precision_0_std = Results_df.groupby('y')['precision 0'].std()
        precision_1_mean = Results_df.groupby('y')['precision 1'].mean()
        precision_1_std = Results_df.groupby('y')['precision 1'].std()
        recall_0_mean = Results_df.groupby('y')['recall 0'].mean()
        recall_0_std = Results_df.groupby('y')['recall 0'].std()
        recall_1_mean = Results_df.groupby('y')['recall 1'].mean()
        recall_1_std = Results_df.groupby('y')['recall 1'].std()
        F1_0_mean = Results_df.groupby('y')['F1 0'].mean()
        F1_0_std = Results_df.groupby('y')['F1 0'].std()
        F1_1_mean = Results_df.groupby('y')['F1 1'].mean()
        F1_1_std = Results_df.groupby('y')['F1 1'].std()
        true_neg_mean = Results_df.groupby('y')['True Neg'].mean()
        true_neg_std = Results_df.groupby('y')['True Neg'].std()
        false_pos_mean = Results_df.groupby('y')['False Pos'].mean()
        false_pos_std = Results_df.groupby('y')['False Pos'].std()        
        false_neg_mean = Results_df.groupby('y')['False Neg'].mean()
        false_neg_std = Results_df.groupby('y')['False Neg'].std()      
        true_pos_mean = Results_df.groupby('y')['True Pos'].mean() 
        true_pos_std = Results_df.groupby('y')['True Pos'].std()   
        

        average_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std,
                                   true_neg_mean, true_neg_std,
                                   false_pos_mean, false_pos_std,
                                   false_neg_mean, false_neg_std,
                                   true_pos_mean, true_pos_std], axis=1)
    
        average_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std', 'true_neg_mean', 'true_neg_std',
                                 'false_pos_mean', 'false_pos_std','false_neg_mean', 'false_neg_std',
                                 'true_pos_mean', 'true_pos_std']                                

        maxacc = average_df[average_df.acc_mean == average_df.acc_mean.max()]
        maxacc_copy  = maxacc.copy()
        maxacc_copy['x_col'] = x
        
        model_columns = model_columns.append(training_columns_list)
        #print(x,y)
    test_index = pd.DataFrame(test_data.index.values)
    test_index_total = pd.concat([test_index_total, test_index],axis = 1)
    test_y = test_y.rename(x)
    prod = prod.append(test_y)
    maxacc_comb = maxacc_comb.append(maxacc_copy)  

total_results = val_tot.T
 
maxacc_comb.to_csv("ChargesJchem.csv")
model_columns = model_columns.rename(columns = {0:'x', 1:'y', 2: 'features'})
model_columns = model_columns.drop_duplicates(subset = ['x','y'])
model_columns.to_csv("ChargesJchem.csv", mode="a")

In [11]:
maxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,true_neg_mean,true_neg_std,false_pos_mean,false_pos_std,false_neg_mean,false_neg_std,true_pos_mean,true_pos_std,x_col
y,,,,,,,,,,,,,,,,,,,,,,,
0.25,0.840476,0.006827,0.920025,0.005333,0.568536,0.015834,0.879412,0.006354,0.675000,0.023570,0.899244,0.004380,0.617082,0.016907,149.500000,1.080123,20.500000,1.080123,13.000000,0.942809,27.000000,0.942809,1
0.25,0.858516,0.019520,0.935461,0.016460,0.605117,0.040388,0.886451,0.009664,0.740000,0.069491,0.910264,0.012012,0.665546,0.051717,150.250000,1.332785,19.250000,1.681947,10.400000,2.779625,29.600000,2.779625,2
0.25,0.843849,0.026592,0.920717,0.025121,0.591210,0.039175,0.880922,0.011722,0.697500,0.083395,0.900312,0.017470,0.639316,0.056769,142.500000,11.211909,19.166667,1.464131,12.100000,3.335804,27.900000,3.335804,3
0.30,0.843312,0.020874,0.919204,0.020900,0.591829,0.030923,0.881852,0.009258,0.690625,0.070866,0.900072,0.013649,0.636782,0.046057,142.025000,9.480040,18.975000,1.349026,12.375000,2.834653,27.625000,2.834653,4
0.30,0.847172,0.020399,0.923502,0.020672,0.592512,0.028572,0.883174,0.009432,0.701500,0.067652,0.902818,0.013524,0.641725,0.042729,145.960000,11.626781,19.240000,1.478554,11.940000,2.706078,28.060000,2.706078,5
0.30,0.858238,0.031333,0.929649,0.023404,0.619476,0.067187,0.891010,0.020103,0.725833,0.082591,0.909852,0.020233,0.667762,0.070822,146.283333,10.642243,17.883333,3.410312,10.966667,3.303653,29.033333,3.303653,6
0.30,0.855089,0.030055,0.927401,0.022352,0.611093,0.065639,0.889521,0.019055,0.714643,0.081260,0.908007,0.019296,0.658222,0.069672,146.771429,9.920220,18.228571,3.284445,11.414286,3.250386,28.585714,3.250386,7
0.25,0.853483,0.029076,0.927958,0.021434,0.604038,0.062413,0.887024,0.018497,0.715313,0.077795,0.906979,0.018738,0.654481,0.066825,147.587500,9.957751,18.787500,3.169050,11.387500,3.111814,28.612500,3.111814,8
0.30,0.849961,0.028411,0.927322,0.020258,0.597330,0.063970,0.882886,0.022035,0.714444,0.076893,0.904436,0.018483,0.649742,0.065182,146.177778,9.887659,19.377778,3.685922,11.422222,3.075740,28.577778,3.075740,9


In [12]:
selected_features = ['Hirshfeld Heavy Atom Charge', 'Hirshfeld Carbon Charge', 'Hirshfeld Hydrogen Charge', 'ESP Heavy Atom Charge', 'ESP Hydrogen Charge',
                     'NPA Hydrogen Charge', 'Mulliken Heavy Charge', 'Mulliken Hydrogen Charge', 'Aliphatic Atom Count', 'Aliphatic Ring Count', 'Aromatic Atom Count', 'Steric Effect Index', 'Balaban Index', 'Chain Atom Count', 'Distance Degree', 'Dreiding Energy', 'Eccentricity', 
                     'Hydrogen Acceptor Count', 'Heteroatom Aromatic Ring Count', 'Hydrogen Donor Count', 'Largest Ring Size', 'Sigma Electronegativity']

#Loads validation dataset for borlation using the final reduced features 
unknownSubstrates=pd.read_csv('validation8-26-24.csv')

# Convert validation substrates Inchi's to Mordred and combine into Dataframe with atomic charges and JChem paramters
New_Substrate = unknownSubstrates['Substrate']
New_Substrate_mol_list = []
for inChi_New_Substrate in New_Substrate:
  New_Substrate_mol = Chem.MolFromInchi(inChi_New_Substrate)
  New_Substrate_mol_list.append(New_Substrate_mol)

New_Substrate_data = []
New_Substrate_data = calc.pandas(New_Substrate_mol_list)
New_Substrate_data = New_Substrate_data.apply(pd.to_numeric, errors='coerce')
New_Substrate_data.fillna(0, inplace=True)                                                                  
XnewSec = pd.concat((unknownSubstrates, New_Substrate_data), axis=1)
Xnew = XnewSec[selected_features]

val_tot = pd.DataFrame()

for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 200,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(data, "grouped", set_numbers)
    test_data = filter_rows_by_values2(data, "grouped", set_numbers)
   
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio' , axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio' , axis = 1)
    test_y = test_data['Product_Ratio']
   
    #Apply over-sampling to training set
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(training_X, training_y)    
    X_train = X_resampled[selected_features]

    #Random Forest Classifier
    rfc = RandomForestClassifier(n_estimators=800,max_depth=9)
    rfc.fit(X_train, y_resampled)
    
    #Evaluate the model on validation set
    ynew = rfc.predict(Xnew)
    validation_prediction_df = pd.DataFrame(ynew, columns = [(x)])
    validation_prediction_df.merge(validation_prediction_df, on=x)
    val_pred_T = validation_prediction_df.T
    val_tot = val_tot.append(val_pred_T)        

#Print the validation evaluations for model
unknownSubstrates_prod = unknownSubstrates['Product_Ratio']
total_val_results_transposed = val_tot.T
Val_results = pd.concat((unknownSubstrates_prod, total_val_results_transposed), axis=1)

#Write the results onto a CSV file 

Val_results.to_csv("ChargesJchem.csv", mode="a")

Val_results

100%|██████████| 81/81 [00:12<00:00,  6.48it/s]


,Product_Ratio,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
76,1,1,0,0,1,0,0,1,1,1,0
77,0,0,0,1,0,0,0,0,1,0,1
78,0,1,0,0,1,0,0,1,1,1,0
79,0,0,0,0,0,0,0,0,0,0,0
